In [38]:
import pandas as pd   
from matplotlib import pyplot as plt
from numpy import mean
import time

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import PolynomialFeatures

import warnings 
warnings.filterwarnings('ignore')


from sklearn.linear_model import LinearRegression

In [39]:
df1 = pd.read_csv("train.csv")
#df1.head()
df2 = pd.read_csv('test.csv')
#df2

In [ ]:
#from df1 drop sub_are
df1 = df1.drop(columns=['sub_area'])


In [ ]:
#from df2 drop sub_area and row ID 
df2 = df2.drop(columns=['sub_area', 'row ID'])

In [ ]:
df2 = df2.drop(columns=[ 'row ID'])

In [40]:
category_frequencies = df1['sub_area'].value_counts()

print(category_frequencies)

Poselenie Shhapovskoe             1105
Alekseevskoe                      1104
Tverskoe                          1101
Poselenie Mosrentgen              1099
Caricyno                          1097
                                  ... 
Posleenie Novofedorovskoe            3
Poseleine Voronovskoe                3
Poselenie Mhiajlovo-Jarcevskoe       3
Prsopekt Vernadskogo                 2
Poselenie Krasnopahorksoe            2
Name: sub_area, Length: 1927, dtype: int64


In [41]:
infrequent_categories = category_frequencies[category_frequencies < 900].index.tolist()
print(len(infrequent_categories))
print(len(category_frequencies))

1781
1927


In [42]:
df1['sub_area'] = df1['sub_area'].replace(infrequent_categories, 'Other')
df2['sub_area'] = df2['sub_area'].replace(infrequent_categories, 'Other')
category_frequencies1 = df1['sub_area'].value_counts()
print(len(category_frequencies1))

147


In [ ]:
df_onehot = pd.get_dummies(df1)
df_onehot.dtypes
test_onehot = pd.get_dummies(df2)

In [43]:
label_encoder = LabelEncoder()
categorical_cols = df1.select_dtypes(include=['object']).columns.tolist()
categorical_colstest = df2.select_dtypes(include=['object']).columns.tolist()

# You can also encode labels to identify categorical columns
for col in categorical_cols:
    df1[col] = label_encoder.fit_transform(df1[col])

for col in categorical_colstest:
    df2[col] = label_encoder.fit_transform(df2[col])


In [ ]:
print

In [ ]:
X = df_onehot.drop(columns=['price_doc'])
print(X.shape)
y = df_onehot[['price_doc']]






In [44]:
X = df1.drop(columns=['price_doc'])
print(X.shape)
y = df1[['price_doc']]






(181507, 271)


In [45]:
scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)
X_testscaled = scaler.fit_transform(test_onehot)

In [46]:
#drop row id from x_testscaled
X_testscaled = X_testscaled[:, 1:]


In [47]:
print(X_testscaled.shape)

(77789, 434)


In [48]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_scaled)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(X_testscaled)


In [50]:
print(X_scaled.shape)

(181507, 271)


In [51]:
X_testscaled = X_testscaled[X_scaled.columns]

In [52]:
from sklearn.decomposition import PCA
n_components = 100  # You can choose the number of components based on your requirements
pca = PCA(n_components=n_components)

# Apply PCA to the training data
X_pca = pca.fit_transform(X_scaled)

# Apply the same PCA transformation to the test data
df2_onehot_pca = pca.transform(X_testscaled)

In [101]:
print(df2_onehot_pca.shape)

(77789, 100)


In [55]:
ols_reg = LinearRegression()
sfs = SequentialFeatureSelector(ols_reg, direction='forward', n_features_to_select=15,n_jobs=-1)
sfs.fit(X_pca, y)
#X_selected = sfs.transform(X_pca)
# Print the names of the selected features
print("Selected Features:", sfs.get_feature_names_out())




Selected Features: ['x0' 'x1' 'x3' 'x4' 'x7' 'x9' 'x15' 'x16' 'x21' 'x22' 'x23' 'x41' 'x70'
 'x74' 'x99']


In [96]:
#save the selected features in a dataframe
X_selected = pd.DataFrame(sfs.transform(X_pca), columns=sfs.get_feature_names_out())

In [120]:
print(X_testscaled.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77789 entries, 0 to 77788
Data columns (total 100 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       77789 non-null  float64
 1   1       77789 non-null  float64
 2   2       77789 non-null  float64
 3   3       77789 non-null  float64
 4   4       77789 non-null  float64
 5   5       77789 non-null  float64
 6   6       77789 non-null  float64
 7   7       77789 non-null  float64
 8   8       77789 non-null  float64
 9   9       77789 non-null  float64
 10  10      77789 non-null  float64
 11  11      77789 non-null  float64
 12  12      77789 non-null  float64
 13  13      77789 non-null  float64
 14  14      77789 non-null  float64
 15  15      77789 non-null  float64
 16  16      77789 non-null  float64
 17  17      77789 non-null  float64
 18  18      77789 non-null  float64
 19  19      77789 non-null  float64
 20  20      77789 non-null  float64
 21  21      77789 non-null  float64
 2

In [121]:
# Create a mapping dictionary for the old column names to the new names
column_mapping = {i: f'x{i}' for i in range(100)}

# Rename the columns in X_testscaled
X_testscaled.rename(columns=column_mapping, inplace=True)

# Get the names of the selected features
selected_feature_names = sfs.get_feature_names_out()

# Filter columns in X_testscaled based on selected features
X_testselected = X_testscaled[selected_feature_names]


In [107]:
#covert x_scaled to dataframe
X_scaled = pd.DataFrame(X_pca)
#convert x_testscaled to dataframe
X_testscaled = pd.DataFrame(df2_onehot_pca)

In [122]:
print(X_testselected.shape)

(77789, 15)


In [ ]:
#give me code for feature importance using linear regression
model = LinearRegression()
model.fit(X_scaled, y)
# get importance
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

# drop features with low importance
threshold = 70000  # Adjust the threshold as needed
selected_features = [feature for feature, score in zip(X_scaled.columns, importance) if score >= threshold]
X_selected = X_scaled[selected_features]

# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()



In [ ]:
import statsmodels.api as sm
import pandas as pd

# Convert scaled_df1 to a DataFrame without specifying columns

# Add a constant term to the feature matrix
X_with_const = sm.add_constant(X_scaled)

# Fit a linear regression model
model = sm.OLS(y, X_with_const).fit()

# Get p-values for each feature
p_values = model.pvalues[1:]  # Exclude the constant term

# Set your desired threshold for p-value
threshold = 0.05

# Filter features based on p-value
selected_features = p_values[p_values < threshold].index

# Display selected features
print("Selected Features:")
print(selected_features)

# Select columns in the DataFrame
X_selected = X_scaled[selected_features]


In [ ]:
# original_feature_indices = np.arange(X.shape[1])
threshold_value = 0.001
variance_filter = VarianceThreshold(threshold=threshold_value)

# Fit and transform the feature matrix
X_filtered = variance_filter.fit_transform(X_pca)

X_test_high_variance = variance_filter.transform(df2_onehot_pca)

In [109]:
print(X_selected.shape)

(181507, 15)


In [123]:
print(X_testscaled.shape)

(77789, 100)


In [111]:
print(X_selected.head())

          x0        x1         x3         x4        x7        x9       x15  \
0 -25.097684  2.081439  -2.397080  -0.705120  2.641445  2.644747 -1.254887   
1 -24.989207  1.837098  -1.051256  -0.960814  2.118892  3.747045 -0.075839   
2 -25.501817  2.397034  -3.184963  -0.549944  1.964646  0.897015 -0.449329   
3  -8.200355 -8.415134  33.080799 -11.824804 -4.063293  2.374556  2.726748   
4 -23.334034  1.272442   0.247319  -3.555951  3.623278 -2.094094 -0.177176   

        x16       x21       x22       x23       x41       x70       x74  \
0  1.351893  0.981178  1.147479  0.986381 -0.319262  0.267783 -0.654090   
1  0.781967 -0.131474  1.158358  0.479678  0.555907 -0.513117  0.067374   
2  0.491925  0.023041 -0.200901  0.678492  0.711441 -0.224848  0.290627   
3 -0.601125 -2.082962 -1.014878  1.976011  0.911945  0.575252  1.412980   
4  0.290085 -3.074768 -4.063173  0.199125 -0.025165  0.362275  0.179363   

        x99  
0 -0.039765  
1  0.072293  
2 -1.048838  
3  1.596842  
4 -0.77861

In [ ]:
print(X_filtered.shape)

In [ ]:
print(X_test_high_variance.shape)

In [ ]:
#store x_filtered as a df
X_filtered = pd.DataFrame(X_filtered)

In [63]:
X_selected = pd.DataFrame(X_selected)

In [115]:
#keep those columns in X_testscaled that are in X_selected
X_testscaled = X_testscaled[X_selected.columns]

KeyError: "None of [Index(['x0', 'x1', 'x3', 'x4', 'x7', 'x9', 'x15', 'x16', 'x21', 'x22', 'x23',\n       'x41', 'x70', 'x74', 'x99'],\n      dtype='object')] are in the [columns]"

In [78]:
# Make sure column names are trimmed

# Find the common columns
common_columns = X_testscaled.columns.intersection(X_selected.columns)

# Keep only the common columns in X_testscaled
X_testscaled_filtered = X_testscaled[common_columns]

# Now X_testscaled_filtered contains only the columns that are in both X_testscaled and X_selected


In [82]:
common_columns = X_testscaled.columns.intersection(X_selected.columns)
X_testscaled = X_testscaled[common_columns]


In [83]:
print(X_testscaled.shape)


(77789, 0)


In [117]:
# Get the common column names between X_testselected and X_selected
common_columns = X_testscaled.columns.intersection(X_selected.columns)

# Keep only the common columns in X_testselected
X_testselected = X_testscaled[X_selected.columns.intersection(common_columns)]


In [118]:
print(X_testselected.shape)

(77789, 0)


In [71]:
print(X_testscaled_filtered.head())

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_selected)
test_scaled = scaler.transform(test_onehot)
print(test_scaled.shape)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
print("Training Set Info:")
print(X.info())

print("\nTest Set Info:")
print(test_onehot.info())


In [125]:
poly = PolynomialFeatures(3, interaction_only=True)
X2 = poly.fit_transform(X_selected)
test2 = poly.fit_transform(X_testselected)
print(X2.shape)
print(poly.get_feature_names_out())
reg2 = LinearRegression().fit(X2, y)
print(reg2.score(X2, y))
print(reg2.coef_)
print(reg2.intercept_)
md_probs = reg2.predict(test2)

(181507, 816)
['1' 'x0' 'x1' 'x3' 'x4' 'x7' 'x9' 'x15' 'x16' 'x21' 'x22' 'x23' 'x41'
 'x70' 'x74' 'x99' 'x0^2' 'x0 x1' 'x0 x3' 'x0 x4' 'x0 x7' 'x0 x9' 'x0 x15'
 'x0 x16' 'x0 x21' 'x0 x22' 'x0 x23' 'x0 x41' 'x0 x70' 'x0 x74' 'x0 x99'
 'x1^2' 'x1 x3' 'x1 x4' 'x1 x7' 'x1 x9' 'x1 x15' 'x1 x16' 'x1 x21'
 'x1 x22' 'x1 x23' 'x1 x41' 'x1 x70' 'x1 x74' 'x1 x99' 'x3^2' 'x3 x4'
 'x3 x7' 'x3 x9' 'x3 x15' 'x3 x16' 'x3 x21' 'x3 x22' 'x3 x23' 'x3 x41'
 'x3 x70' 'x3 x74' 'x3 x99' 'x4^2' 'x4 x7' 'x4 x9' 'x4 x15' 'x4 x16'
 'x4 x21' 'x4 x22' 'x4 x23' 'x4 x41' 'x4 x70' 'x4 x74' 'x4 x99' 'x7^2'
 'x7 x9' 'x7 x15' 'x7 x16' 'x7 x21' 'x7 x22' 'x7 x23' 'x7 x41' 'x7 x70'
 'x7 x74' 'x7 x99' 'x9^2' 'x9 x15' 'x9 x16' 'x9 x21' 'x9 x22' 'x9 x23'
 'x9 x41' 'x9 x70' 'x9 x74' 'x9 x99' 'x15^2' 'x15 x16' 'x15 x21' 'x15 x22'
 'x15 x23' 'x15 x41' 'x15 x70' 'x15 x74' 'x15 x99' 'x16^2' 'x16 x21'
 'x16 x22' 'x16 x23' 'x16 x41' 'x16 x70' 'x16 x74' 'x16 x99' 'x21^2'
 'x21 x22' 'x21 x23' 'x21 x41' 'x21 x70' 'x21 x74' 'x21 x99' 'x

In [126]:
additional_data = pd.read_csv("test.csv")

# Create a DataFrame with 'row ID' and predicted values
predicted_df = pd.DataFrame({
    'row ID': additional_data['row ID'],
    'price_doc': md_probs[:,0]  # Replace with the actual column name for your predictions
})

# Save the DataFrame to a new CSV file
predicted_df.to_csv('predictions26.csv', index=False)